In [ ]:
# 0. Dataset Path Configuration
# ---
import os
# If a 'dataset.csv' file exists in the working directory (e.g. evaluated in GitHub Actions or Colab), use it.
if os.path.exists('dataset.csv'):
    DATA_PATH = 'dataset.csv'
    print(f"Found local dataset at {DATA_PATH}")

In [ ]:
!pip install pathway bokeh panel pandas numpy scikit-learn matplotlib --quiet


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathway as pw
import bokeh.plotting as bkp
import panel as pn

from datetime import timedelta
from sklearn.preprocessing import OneHotEncoder


In [ ]:
pn.extension('bokeh')

In [ ]:
# ---
# 2. Load and Prepare Data
# ---
def load_and_prepare(path):
    df = pd.read_csv(path)
    # Combine date/time into Timestamp
    if 'LastUpdatedDate' in df.columns and 'LastUpdatedTime' in df.columns:
        df['Timestamp'] = pd.to_datetime(
            df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
            format='%d-%m-%Y %H:%M:%S'
        )
    elif 'timestamp' in df.columns:
        df['Timestamp'] = pd.to_datetime(df['timestamp'])
    else:
        raise ValueError("No timestamp columns found.")
    df = df.sort_values('Timestamp').reset_index(drop=True)
    return df

# Load
df = load_and_prepare(DATA_PATH)
print(df.shape)
df.head()

In [ ]:
# ---
# 3. Extensive EDA
# ---
# 3.1 Missing Values
print(df.isnull().sum())

# 3.2 Descriptive Stats
display(df.describe(include='all'))

# 3.3 Temporal Trend: Occupancy Ratio
plt.figure(figsize=(10,4))
plt.scatter(df['Timestamp'], df['Occupancy']/df['Capacity'], alpha=0.3)
plt.title('Occupancy Ratio Over Time')
plt.xlabel('Time'); plt.ylabel('Occupancy Ratio')
plt.show()

# 3.4 Spatial Plot
if {'Latitude','Longitude'}.issubset(df.columns):
    plt.figure(figsize=(6,6))
    sc = plt.scatter(df['Longitude'], df['Latitude'],
                     c=df['Occupancy']/df['Capacity'], cmap='viridis')
    plt.colorbar(sc, label='Occupancy Ratio')
    plt.title('Spatial Occupancy Ratio')
    plt.show()

# 3.5 Correlation Matrix
num_cols = df.select_dtypes(include=[np.number])
corr = num_cols.corr()
display(corr)


In [ ]:
# ---
# 4. Feature Engineering
# ---
def engineer_features(df):
    df2 = df.copy()
    df2['occ_ratio'] = df2['Occupancy'] / df2['Capacity']
    df2['occ_roll3'] = df2['occ_ratio'].rolling(3, min_periods=1).mean()
    if 'SpecialDay' in df2.columns:
        df2['is_special'] = df2['SpecialDay'].astype(int)
    if 'VehicleType' in df2.columns:
        enc = OneHotEncoder(sparse_output=False, drop='first')
        arr = enc.fit_transform(df2[['VehicleType']])
        cols = enc.get_feature_names_out(['VehicleType'])
        df2[cols] = arr
    return df2

# Apply
df_feat = engineer_features(df)
df_feat.head()

In [ ]:
# ---
# 5. Model 1: Baseline Linear Pricing
# ---
def model_baseline(df, alpha=0.1, base_price=10.0):
    df2 = df.copy().reset_index(drop=True)
    df2['price_baseline'] = base_price
    for i in range(1, len(df2)):
        df2.loc[i, 'price_baseline'] = (
            df2.loc[i-1, 'price_baseline'] + alpha * df2.loc[i, 'occ_ratio']
        )
    return df2

df_base = model_baseline(df_feat)

df_base[['Timestamp','price_baseline']].head()

# ---
# 6. Model 2: Demand‑Based Pricing
# ---
def model_demand(df, coeffs, base_price=10.0, λ=1.0):
    df2 = df.copy()
    d = coeffs['α'] * df2['occ_ratio']
    if 'QueueLength' in df2: d += coeffs['β'] * df2['QueueLength']
    if 'TrafficLevel' in df2: d += coeffs['γ'] * df2['TrafficLevel']
    if 'is_special' in df2:   d += coeffs['δ'] * df2['is_special']
    dn = (d - d.min()) / (d.max() - d.min() + 1e-6)
    df2['price_demand'] = base_price * (1 + λ * dn)
    return df2

coeffs = {'α':1.0,'β':0.5,'γ':0.2,'δ':0.3}
df_demand = model_demand(df_feat, coeffs)

df_demand[['Timestamp','price_demand']].head()

# ---
# 7. Model 3: Competitive‑Aware Pricing
# ---
def model_competitive(df, comp_prices, weight=0.5):
    df2 = df.copy()
    df2['comp_price'] = comp_prices
    df2['price_competitive'] = df2['price_demand'] + weight * (df2['comp_price'] - df2['price_demand'])
    return df2

# Random competitor prices example
df_comp = model_competitive(df_demand, np.random.uniform(8,12,len(df_demand)))
df_comp[['Timestamp','price_competitive']].head()

In [ ]:
# ---
# 8. Pathway Real‑Time Streaming Simulation (Colab Only)
# ---
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    import datetime
    import pathway as pw

    df[['Timestamp','Occupancy','Capacity']].to_csv('parking_stream.csv', index=False)

    class ParkingSchema(pw.Schema):
        Timestamp: str
        Occupancy: int
        Capacity: int

    data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=500)

    fmt = "%Y-%m-%d %H:%M:%S"
    data_with_time = data.with_columns(
        t = data.Timestamp.dt.strptime(fmt),
        day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
    )

    delta_window = (
        data_with_time.windowby(
            pw.this.t,
            instance=pw.this.day,
            window=pw.temporal.tumbling(datetime.timedelta(days=1)),
            behavior=pw.temporal.exactly_once_behavior()
        )
        .reduce(
            t = pw.this._pw_window_end,
            occ_max = pw.reducers.max(pw.this.Occupancy),
            occ_min = pw.reducers.min(pw.this.Occupancy),
            cap     = pw.reducers.max(pw.this.Capacity)
        )
        .with_columns(
            price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
        )
    )

In [ ]:
import bokeh.plotting
import panel as pn
pn.extension()

def price_plotter(source):
  fig = bokeh.plotting.figure(
            height=400,
            width=800,
            title="Pathway: Daily Parking Price",
            x_axis_type="datetime"
        )
  fig.line("t", "price", source=source, line_width=2, color="navy")
  fig.circle("t", "price", source=source, size=6, color="red")
  return fig


viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

In [ ]:
# Final cell: run the pipeline
%%capture --no-display
pw.run()